In [2]:
import sqlite3
import json

def load_tables(db_path):
    """Load all tables into a dict of {table_name: [rows as dicts]}"""
    conn = sqlite3.connect(db_path)
    conn.row_factory = sqlite3.Row
    cur = conn.cursor()

    tables = {}
    cur.execute("SELECT name FROM sqlite_master WHERE type='table'")
    for (table_name,) in cur.fetchall():
        cur.execute(f"SELECT * FROM {table_name}")
        rows = [dict(r) for r in cur.fetchall()]
        tables[table_name] = rows

    conn.close()
    return tables

def build_tree(table_name, row, tables, fk_depth=0, child_depth=0, visited=None):
    """
    Build object tree with controlled depth expansion.

    - fk_depth: depth for foreign key expansion (limit = 1).
    - child_depth: depth for implicit parent-child expansion (limit = 3).
    - visited: set to prevent circular recursion.
    """
    if visited is None:
        visited = set()

    obj = dict(row)
    obj["_table"] = table_name  # keep schema info

    row_id = (table_name, row.get("name"))
    if row_id in visited:
        return obj
    visited.add(row_id)

    # -------------------------------
    # 1. Handle foreign key references (limit 1 level)
    # -------------------------------
    if fk_depth < 1:
        for field, value in row.items():
            if field.endswith("Name") and isinstance(value, str):
                ref_table = field[:-4]  # drop "Name" suffix
                if ref_table in tables:
                    ref_row = next((r for r in tables[ref_table] if r.get("name") == value), None)
                    if ref_row:
                        obj[field] = build_tree(
                            ref_table, ref_row, tables,
                            fk_depth=fk_depth + 1,
                            child_depth=child_depth,
                            visited=visited.copy()
                        )

    # -------------------------------
    # 2. Handle implicit children (limit 3 levels)
    # -------------------------------
    if child_depth < 3:
        for t_name, t_rows in tables.items():
            children = [
                build_tree(
                    t_name, child_row, tables,
                    fk_depth=0,  # reset FK depth for children
                    child_depth=child_depth + 1,
                    visited=visited.copy()
                )
                for child_row in t_rows
                if child_row.get("parentSchemaName") == table_name
                and child_row.get("parent") == row.get("name")
            ]
            if children:
                obj.setdefault(t_name, []).extend(children)

    return obj

def build_from_db(db_path, start_table, row_name=None):
    """Entry point: build tree for given table and optional row.name filter"""
    tables = load_tables(db_path)
    if start_table not in tables:
        raise ValueError(f"Table {start_table} not found in {db_path}")

    # pick row
    if row_name:
        row = next((r for r in tables[start_table] if r.get("name") == row_name), None)
        if not row:
            raise ValueError(f"Row with name={row_name} not found in {start_table}")
    else:
        row = tables[start_table][0]  # first row if not specified

    return build_tree(start_table, row, tables)

if __name__ == "__main__":
    db_path = "demo.db"
    start_table = "SalesInvoice"  # change this as needed
    row_name = None  # e.g. "INV-0001"

    tree = build_from_db(db_path, start_table, row_name)
    print(json.dumps(tree, indent=2))


{
  "name": "SINV-1001",
  "numberSeries": "SINV-",
  "party": "Daniel",
  "account": "Debtors",
  "date": "2024-09-02T04:00:00.000Z",
  "priceList": null,
  "netTotal": "48367.00000000000",
  "baseGrandTotal": "55183.42000000000",
  "grandTotal": "55183.42000000000",
  "setDiscountAmount": 0,
  "discountAmount": "0.00000000000",
  "discountPercent": 0.0,
  "entryCurrency": "Party",
  "currency": "INR",
  "exchangeRate": 1.0,
  "discountAfterTax": 0,
  "makeAutoPayment": 0,
  "makeAutoStockTransfer": 0,
  "outstandingAmount": "0.00000000000",
  "stockNotTransferred": 0.0,
  "terms": "",
  "attachment": null,
  "isReturned": 0,
  "isFullyReturned": 0,
  "isSyncedWithErp": 0,
  "backReference": null,
  "returnAgainst": null,
  "quote": null,
  "loyaltyProgram": null,
  "redeemLoyaltyPoints": 0,
  "loyaltyPoints": 0,
  "isPOS": 0,
  "isPricingRuleApplied": 0,
  "createdBy": "i771468@gmail.com",
  "modifiedBy": "i771468@gmail.com",
  "created": "2025-08-05T15:57:40.569Z",
  "modified": "20